In [1]:
!unzip -qq '/content/drive/MyDrive/dataset.zip' -d './dataset'

In [2]:
import os

original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

## 데이터 정리를 위한 목록 및 폴더 생성

In [3]:
import shutil

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

## 데이터 현황 확인

In [4]:
import math

for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print('Train size(',cls,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',cls,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
    
    test_fnames = fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
    print('Test size(',cls,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  307
Validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Train size( Potato___Late_blight ):  600
Validation size( Potato___Late_blight ):  200
Test size( Potato___Late_blight ):  200
Train size( Tomato___Septoria_leaf_spot ):  1062
Validation size( Tomato___Septoria_leaf_spot ):  354
Test size( Tomato___Septoria_leaf_spot ):  354
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Peach___Bacterial_spot ):  1378
Validation size( Peach___Bacterial_spot ):  459
Test size( Peach___Bacterial_spot ):  459
Train size( Tomato___Bacterial_spot ):  1276
Validation size( Tomato___Bacterial_spot ):  425
Test size( Tomato___Bacterial_spot ):  425
Train size( Apple

## 학습 준비

In [5]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
BATCH_SIZE = 256 # 크게 할수록 학습이 빨리 되나, GPU 메모리 한계가 있음
EPOCH = 30

In [6]:
import torchvision.transforms as transforms

data_transforms = {
    'train': transforms.Compose([transforms.Resize([64, 64]), # 전처리
        transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), # 상하좌우로 이미지 반전 -> 기하학적 증강 -> 과적합 방지용
        transforms.RandomCrop(52), transforms.ToTensor(), # Crop: 이미지 자르기 -> 이미지의 일부를 보여주는 효과 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]), # RGB 색깔이 MinMax-Scailing이 된것 (평균, 표준편차)
    
    'val': transforms.Compose([transforms.Resize([64, 64]),
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [7]:
from torchvision.datasets import ImageFolder

data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x),
                                 transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 모델 불러오기 - resnet50

In [8]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models # 모델들을 불러올 수 있다
from torch.optim import lr_scheduler

resnet = models.resnet50(pretrained=True) # pretained=True: 학습이 된 가중치를 가져옴, False는 model의 구조만
num_ftrs = resnet.fc.in_features # 마지막 layer의 채널 수
resnet.fc = nn.Linear(num_ftrs, 33) # 클래스의 수가 resnet50과 다름 -> 출력단의 채널 수를 수정
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001) # p.requires_grad: 마지막에 교체한 33개의 fully-connected-layer는 학습이 안되어 있으므로 설정

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # epoch에 따라 learning rate를 수정 -> 7epoch마다 learning rate를 0.1씩 감소시킴

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
ct = 0
for child in resnet.children(): # resnet.children(): 어떤 모델에 속해있는 하위 layer들의 정보를 받아옴, resnet은 10개의 layer가 존재 0~5 layer들을 학습하지 않도록 -> 입력에 가까운 층들
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [11]:
import copy
import time

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('-------------- epoch {} --------------'.format(epoch+1))
        since = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # 만약에 train모드라면 gradient 학습을 허가
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # 원-핫 인코딩된 33개의 output들 중 가장 높은 인덱스 값을 preds에 저장
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0) # loss 업데이트, inputs.size(0)은 batch size, loss는 batch size 개수만큼 곱해진 값의 총 합산
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

## 학습 시작

In [12]:
model_restnet50 = train_resnet(resnet, criterion, optimizer_ft,
                               exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_restnet50, 'resnet50.pt')

-------------- epoch 1 --------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.5766 Acc: 0.8229
val Loss: 0.3364 Acc: 0.8954
Completed in 1m 12s
-------------- epoch 2 --------------
train Loss: 0.2204 Acc: 0.9296
val Loss: 0.2023 Acc: 0.9364
Completed in 1m 12s
-------------- epoch 3 --------------
train Loss: 0.1767 Acc: 0.9432
val Loss: 0.1764 Acc: 0.9425
Completed in 1m 13s
-------------- epoch 4 --------------
train Loss: 0.1356 Acc: 0.9547
val Loss: 0.1145 Acc: 0.9656
Completed in 1m 19s
-------------- epoch 5 --------------
train Loss: 0.1237 Acc: 0.9591
val Loss: 0.1411 Acc: 0.9563
Completed in 1m 20s
-------------- epoch 6 --------------
train Loss: 0.1003 Acc: 0.9671
val Loss: 0.1549 Acc: 0.9538
Completed in 1m 19s
-------------- epoch 7 --------------
train Loss: 0.0941 Acc: 0.9691
val Loss: 0.1281 Acc: 0.9624
Completed in 1m 18s
-------------- epoch 8 --------------
train Loss: 0.0491 Acc: 0.9840
val Loss: 0.0500 Acc: 0.9855
Completed in 1m 17s
-------------- epoch 9 --------------
train Loss: 0.0275 Acc: 0.9910
val Loss: 0.0461 Acc: 0.9

## 테스트셋 적용

In [14]:
transform_resNet = transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 모델 평가

In [15]:
import torch.nn.functional as F

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [16]:
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc: ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ResNet test acc:  99.04869195143323
